In [1]:
import pandas as pd
from tableone import TableOne, load_dataset

In [2]:
train = pd.read_csv("~/project/n2c2/2022/Data/train.csv")
dev = pd.read_csv("~/project/n2c2/2022/Data/dev.csv")
test = pd.read_csv("/home/vs428/project/n2c2/2022/Data/n2c2_track3_test.csv")

In [3]:
data = pd.concat([train,dev, test])

In [4]:
import jsonlines

In [ ]:
with jsonlines.open("/home/vs428/project/n2c2

In [5]:
train.shape, dev.shape, test.shape

((4633, 5), (597, 5), (667, 5))

In [6]:
total = train.shape[0] + dev.shape[0] +  test.shape[0]
print(f"Train ratio: {train.shape[0]/total}")
print(f"Dev ratio: {dev.shape[0]/total}")
print(f"Test ratio: {test.shape[0]/total}")

Train ratio: 0.7856537222316432
Dev ratio: 0.10123791758521282
Test ratio: 0.11310836018314398


In [22]:
data

,ROW ID,HADM ID,Assessment,Plan Subsection,Relation
0,701359,186454,"51 yr old F with a history of 3V CAD, confirme...",# CORONARIES: Patient with 3 vessel disease on...,Direct
1,701359,186454,"51 yr old F with a history of 3V CAD, confirme...",# Acute on Chronic systolic CHF: Patient with ...,Indirect
2,701359,186454,"51 yr old F with a history of 3V CAD, confirme...",# Social stressors: Recently lost mother to ca...,Neither
3,701359,186454,"51 yr old F with a history of 3V CAD, confirme...","# FEN: Replete lytes PRN, NPO for CABG today\n...",Not Relevant
4,554144,196435,"CHRONIC OBSTRUCTIVE PULMONARY DISEASE (COPD, B...",# Dyspnea: Differential in this patient would ...,Direct
...,...,...,...,...,...
662,560683,196558,"DEEP VENOUS THROMBOSIS (DVT), UPPER EXTREMITY\...",5) HTN: Cont lisinopril,Neither
663,560683,196558,"DEEP VENOUS THROMBOSIS (DVT), UPPER EXTREMITY\...",6) CRI: Last Cr 1.2: Hold naproxen given...,Neither
664,560683,196558,"DEEP VENOUS THROMBOSIS (DVT), UPPER EXTREMITY\...","7) Nausea: likely chemo SE, cont compazi...",Neither
665,560683,196558,"DEEP VENOUS THROMBOSIS (DVT), UPPER EXTREMITY\...","8) Hyperglycemia: trend, consider HISS",Neither


In [5]:
pts = pd.read_csv("/home/vs428/project/MIMIC/files/mimiciii/1.4/PATIENTS.csv")

In [6]:
pts

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...
46515,31840,44089,M,2026-05-25 00:00:00,NaN,NaN,NaN,0
46516,31841,44115,F,2124-07-27 00:00:00,NaN,NaN,NaN,0
46517,31842,44123,F,2049-11-26 00:00:00,2135-01-12 00:00:00,2135-01-12 00:00:00,NaN,1
46518,31843,44126,F,2076-07-25 00:00:00,NaN,NaN,NaN,0


In [7]:
hadms = pd.read_csv("/home/vs428/project/MIMIC/files/mimiciii/1.4/ADMISSIONS.csv")

In [8]:
diags = pd.read_csv("/home/vs428/project/MIMIC/files/mimiciii/1.4/DIAGNOSES_ICD.csv")

In [9]:
icd_map = pd.read_csv("/home/vs428/project/MIMIC/files/mimiciii/1.4/D_ICD_DIAGNOSES.csv")

In [10]:
# set(icd_map['ICD9_CODE'].str.slice(0,3))

In [11]:
primary_diags = diags[diags['SEQ_NUM'] == 1]

In [12]:
primary_diags

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
14,1311,109,173633,1.0,40301
28,1488,112,174105,1.0,53100
33,1493,113,109976,1.0,1915
36,1496,114,178393,1.0,41401
...,...,...,...,...,...
650989,639745,97488,152542,1.0,566
651000,639756,97488,161999,1.0,43411
651024,639780,97492,189314,1.0,34680
651032,639788,97497,168949,1.0,0529


In [26]:
pt_ids = hadms.merge(data, how="inner", left_on="HADM_ID", right_on="HADM ID")
# pt_ids = data.merge(hadms, how="left", right_on="HADM_ID", left_on="HADM ID", indicator=True)

In [27]:
with pd.option_context("display.max_columns", None):
    display(pt_ids)

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,ROW ID,HADM ID,Assessment,Plan Subsection,Relation
0,390,305,194340,2129-08-20 18:03:00,2129-09-07 17:55:00,2129-09-07 17:55:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,PORT,CATHOLIC,MARRIED,WHITE,NaN,NaN,HEART FAILURE,1,1,560683,194340,ssessment and Plan\n SEPSIS WITHOUT ORGAN DYSF...,#. Sepsis: Patient has re-developed fevers on ...,Direct
1,390,305,194340,2129-08-20 18:03:00,2129-09-07 17:55:00,2129-09-07 17:55:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,PORT,CATHOLIC,MARRIED,WHITE,NaN,NaN,HEART FAILURE,1,1,560683,194340,ssessment and Plan\n SEPSIS WITHOUT ORGAN DYSF...,#. Volume status/Chronic systolic HF- As evide...,Neither
2,390,305,194340,2129-08-20 18:03:00,2129-09-07 17:55:00,2129-09-07 17:55:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,PORT,CATHOLIC,MARRIED,WHITE,NaN,NaN,HEART FAILURE,1,1,560683,194340,ssessment and Plan\n SEPSIS WITHOUT ORGAN DYSF...,#. Peripheral arterial disease: will follow up...,Neither
3,390,305,194340,2129-08-20 18:03:00,2129-09-07 17:55:00,2129-09-07 17:55:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,PORT,CATHOLIC,MARRIED,WHITE,NaN,NaN,HEART FAILURE,1,1,560683,194340,ssessment and Plan\n SEPSIS WITHOUT ORGAN DYSF...,#. Insulin dependent diabetes: continue ISS wi...,Neither
4,390,305,194340,2129-08-20 18:03:00,2129-09-07 17:55:00,2129-09-07 17:55:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,PORT,CATHOLIC,MARRIED,WHITE,NaN,NaN,HEART FAILURE,1,1,560683,194340,ssessment and Plan\n SEPSIS WITHOUT ORGAN DYSF...,#. Hypertension: hold antihypertensives while ...,Neither
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5892,58581,98755,184156,2167-05-17 12:36:00,2167-05-18 17:03:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,LONG TERM CARE HOSPITAL,Medicare,ENGL,JEWISH,WIDOWED,WHITE,2167-05-17 10:48:00,2167-05-17 14:02:00,ASTHMA/COPD EXACERBATION,0,1,573914,184156,Assessment and Plan: This is a 80 yo female wi...,# Hypoxia/Hypercarbia: likely in the setting o...,Direct
5893,58581,98755,184156,2167-05-17 12:36:00,2167-05-18 17:03:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,LONG TERM CARE HOSPITAL,Medicare,ENGL,JEWISH,WIDOWED,WHITE,2167-05-17 10:48:00,2167-05-17 14:02:00,ASTHMA/COPD EXACERBATION,0,1,573914,184156,Assessment and Plan: This is a 80 yo female wi...,# Diastolic HF: doesn't seem to be in overt he...,Indirect
5894,58581,98755,184156,2167-05-17 12:36:00,2167-05-18 17:03:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,LONG TERM CARE HOSPITAL,Medicare,ENGL,JEWISH,WIDOWED,WHITE,2167-05-17 10:48:00,2167-05-17 14:02:00,ASTHMA/COPD EXACERBATION,0,1,573914,184156,Assessment and Plan: This is a 80 yo female wi...,# DM2: not on home medications; fingersticks h...,Indirect
5895,58581,98755,184156,2167-05-17 12:36:00,2167-05-18 17:03:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,LONG TERM CARE HOSPITAL,Medicare,ENGL,JEWISH,WIDOWED,WHITE,2167-05-17 10:48:00,2167-05-17 14:02:00,ASTHMA/COPD EXACERBATION,0,1,573914,184156,Assessment and Plan: This is a 80 yo female wi...,# Depression/Anxiety: cont home antidepressant...,Neither


In [28]:
pt_ids.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA', 'ROW ID', 'HADM ID',
       'Assessment', 'Plan Subsection', 'Relation'],
      dtype='object')

In [29]:
pt_ids = pt_ids.astype({
    'ROW_ID':int, 
    'SUBJECT_ID':int,
    'HADM_ID':int,
    'ADMITTIME':"datetime64[ns]",
    'DISCHTIME':"datetime64[ns]",
    'DEATHTIME':"datetime64[ns]",
    "EDREGTIME":"datetime64[ns]",
    "EDOUTTIME":"datetime64[ns]",
    'HOSPITAL_EXPIRE_FLAG':int,
    'HAS_CHARTEVENTS_DATA':int,
    'ROW ID':int,
    'HADM ID':int,
})

In [30]:
pt_ids.columns, primary_diags.columns, pts.columns

(Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
        'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
        'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
        'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
        'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA', 'ROW ID', 'HADM ID',
        'Assessment', 'Plan Subsection', 'Relation'],
       dtype='object'),
 Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'], dtype='object'),
 Index(['ROW_ID', 'SUBJECT_ID', 'GENDER', 'DOB', 'DOD', 'DOD_HOSP', 'DOD_SSN',
        'EXPIRE_FLAG'],
       dtype='object'))

In [31]:
pt_ids.shape

(5897, 24)

In [147]:
pt_ids_with_diags = pt_ids.merge(primary_diags[['HADM_ID', "ICD9_CODE"]], on="HADM_ID", how="inner")

In [148]:
pt_ids_with_diags = pt_ids_with_diags.merge(pts[['SUBJECT_ID', 'GENDER', 'DOB']], on="SUBJECT_ID", how='inner')

In [149]:
pt_ids_with_diags.columns, icd_map.columns

(Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
        'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
        'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
        'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
        'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA', 'ROW ID', 'HADM ID',
        'Assessment', 'Plan Subsection', 'Relation', 'ICD9_CODE', 'GENDER',
        'DOB'],
       dtype='object'),
 Index(['ROW_ID', 'ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE'], dtype='object'))

In [150]:
icd_map[icd_map['ICD9_CODE'] == "0389"]

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
609,660,0389,Septicemia NOS,Unspecified septicemia


In [151]:
pt_ids_with_diags.sample(10)

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,ROW ID,HADM ID,Assessment,Plan Subsection,Relation,ICD9_CODE,GENDER,DOB
5459,58454,98347,110484,2121-10-27 12:30:00,2121-11-06 17:43:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicaid,...,0,1,700410,110484,"49 yo man with presumed CAD, DM who presents w...",Aspiration PNA - Febrile with WBC count elevat...,Direct,99672,M,2073-09-17 00:00:00
5799,57554,95611,105027,2125-09-18 06:22:00,2125-10-08 17:35:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,...,0,1,493359,105027,37 year old female with acute sepsis secondary...,Sinus Tachycardia: Peaked to the 140s yesterda...,Direct,59381,F,2088-06-30 00:00:00
2572,44871,55881,192318,2130-03-21 19:58:00,2130-03-23 14:37:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,...,0,1,560683,192318,Assessment and Plan Mr. [**Known lastname 512*...,# Hypertension. Patient hypertensive on floor ...,Indirect,9351,M,2070-07-15 00:00:00
1953,41310,44890,150042,2129-05-21 16:31:00,2129-05-28 12:29:00,2129-05-28 12:29:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,...,1,1,467544,150042,"The patient is a 73 yo man with h/o PVD, DM, C...",Non-AG metabolic acidosis: Most likely RTA as ...,Neither,27652,M,2055-10-28 00:00:00
5170,53814,83653,147696,2181-04-30 16:52:00,2181-05-08 17:39:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,SNF,Medicare,...,0,1,734034,147696,Mr. [**Known lastname 12424**] is a [**Age ove...,"Tophaceous gout: On chronic steroids, allopuri...",Neither,4260,M,1881-04-30 00:00:00
2100,41599,45801,101382,2180-08-24 19:17:00,2180-08-29 05:04:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,...,0,1,694549,101382,"67 y/o blind, deaf male w/ESRD, s/p cath on [*...",Hypotension: Currently on dopamine given perip...,Direct,41071,M,2112-12-30 00:00:00
3117,47791,64911,103718,2106-10-03 17:59:00,2106-10-12 13:49:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,...,0,1,560683,103718,"57 y/oM with h/o HCV and Alcohol abuse, admitt...",ICU Care\n Nutrition:\n Glycemic Control:\...,Not Relevant,5770,M,2049-06-29 00:00:00
2740,41408,45213,197986,2106-01-31 22:50:00,2106-02-07 12:50:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,...,0,1,715731,197986,41 year old female with strange and complicate...,"# HTN: Well controlled, continue home regimen",Neither,4232,F,2064-07-04 00:00:00
750,18196,14873,197497,2175-05-10 09:38:00,2175-05-26 17:26:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,...,0,1,378689,197497,66 yo woman with ESRD on HD with recent line s...,# Sepsis/fever: Hypotension resolved; no leuko...,Direct,99931,F,2108-05-17 00:00:00
1562,41055,44083,198330,2112-05-28 15:45:00,2112-06-07 16:50:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,...,0,1,580332,198330,"54-yo man with hypertension, dyslipidemia, loc...",# Pericardial effusion: Patient is s/p pericar...,Direct,4239,M,2057-11-15 00:00:00


In [152]:
pt_ids_with_diags = pt_ids_with_diags.merge(icd_map[['ICD9_CODE', "LONG_TITLE"]], on="ICD9_CODE", how="inner")

In [153]:
pt_ids_with_diags = pt_ids_with_diags.astype({
    'GENDER':str,
    'DOB':"datetime64[ns]",
    'ICD9_CODE':str,
    "LONG_TITLE":str
})

In [154]:
pt_ids_with_diags = pt_ids_with_diags.rename({"LONG_TITLE":"PRIMARY_DIAGNOSIS"}, axis=1)

In [155]:
pt_ids_with_diags['ICD_CATEGORY'] = pt_ids_with_diags['ICD9_CODE'].str.slice(0,2)

In [156]:
pt_ids_with_diags = pt_ids_with_diags.merge(icd_map[['ICD9_CODE', "LONG_TITLE"]], left_on='ICD_CATEGORY', right_on="ICD9_CODE", how="left")

In [157]:
from collections import Counter

In [158]:
pt_ids_with_diags['FINAL_PRIMARY_DIAGNOSIS'] = pt_ids_with_diags['LONG_TITLE'].fillna(pt_ids_with_diags['PRIMARY_DIAGNOSIS']).tolist()

In [35]:
columns = [
    # 'ADMITTIME', 'DISCHTIME',
    #    'DEATHTIME',
    'ADMISSION_TYPE',
    # 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 
       # 'MARITAL_STATUS', 
    'ETHNICITY',
    
    # "RELIGION",
    # 'EDREGTIME', 'EDOUTTIME', 
    # 'DIAGNOSIS',
       # 'HOSPITAL_EXPIRE_FLAG', 
           "Relation",
    # "ICD_CATEGORY",
'GENDER',
# 'FINAL_PRIMARY_DIAGNOSIS'    
          ]

groupby = 'Relation'

categorical = ['ADMISSION_TYPE',"DISCHARGE_LOCATION", "INSURANCE", "LANGUAGE", 
 "ETHNICITY", 
               # "FINAL_PRIMARY_DIAGNOSIS", 
               "Relation", 'GENDER',    
               # "ICD_CATEGORY",
              ]

In [41]:
pt_ids = pt_ids.merge(pts[['SUBJECT_ID', 'GENDER', 'DOB']], on="SUBJECT_ID", how='inner')

In [42]:
pt_ids.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA', 'ROW ID', 'HADM ID',
       'Assessment', 'Plan Subsection', 'Relation', 'GENDER', 'DOB'],
      dtype='object')

In [43]:
# pt_ids_with_diags.loc[pt_ids_with_diags['LANGUAGE'].isin(["*CHI", "CANT", "MAND", "VIET"]), "LANGUAGE"] =  "East Asian"
# pt_ids_with_diags.loc[pt_ids_with_diags['LANGUAGE'].isin(["ITAL", "SPAN", "PORT"]), "LANGUAGE"] =  "Romance"
# pt_ids_with_diags.loc[pt_ids_with_diags['LANGUAGE'].isin(["ETHI", "HAIT"]), "LANGUAGE"] =  "African"
# pt_ids_with_diags.loc[pt_ids_with_diags['LANGUAGE'].isin(["PERS", "*FAR"]), "LANGUAGE"] =  "Middle Eastern"
# # pt_ids_with_diags.loc[~pt_ids_with_diags['LANGUAGE'].isin(["ENGL", "SPAN"]), "LANGUAGE"] =  "OTHER"

# pt_ids_with_diags.loc[pt_ids_with_diags['ETHNICITY'].isin(["ASIAN", "ASIAN - ASIAN INDIAN", "ASIAN - CHINESE",
#                                                             "ASIAN - FILIPINO", "ASIAN - JAPANESE","ASIAN - OTHER",
#                                                            ]), "ETHNICITY"] =  "ASIAN"
# pt_ids_with_diags.loc[pt_ids_with_diags['ETHNICITY'].isin(["BLACK/AFRICAN AMERICAN", "BLACK/CAPE VERDEAN", "BLACK/HAITIAN", 
#                                                            ]), "ETHNICITY"] =  "BLACK"
# pt_ids_with_diags.loc[pt_ids_with_diags['ETHNICITY'].isin(["HISPANIC OR LATINO", "HISPANIC/LATINO - MEXICAN", 
#                                                            "HISPANIC/LATINO - PUERTO RICAN", "HISPANIC/LATINO - SALVADORAN",  
#                                                            ]), "ETHNICITY"] =  "HISPANIC/LATINO"
# pt_ids_with_diags.loc[pt_ids_with_diags['ETHNICITY'].isin(["WHITE", "WHITE - OTHER EUROPEAN", "WHITE - RUSSIAN",   
#                                                            ]), "ETHNICITY"] =  "WHITE"
# pt_ids_with_diags.loc[pt_ids_with_diags['ETHNICITY'].isin(["PORTUGUESE",   
#                                                            ]), "ETHNICITY"] =  "OTHER"
# pt_ids_with_diags.loc[pt_ids_with_diags['ETHNICITY'].isin(["AMERICAN INDIAN/ALASKA NATIVE",   "AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE"
#                                                            ]), "ETHNICITY"] =  "AMERICAN INDIAN"

In [44]:
pt_ids.loc[pt_ids['LANGUAGE'].isin(["*CHI", "CANT", "MAND", "VIET"]), "LANGUAGE"] =  "East Asian"
pt_ids.loc[pt_ids['LANGUAGE'].isin(["ITAL", "SPAN", "PORT"]), "LANGUAGE"] =  "Romance"
pt_ids.loc[pt_ids['LANGUAGE'].isin(["ETHI", "HAIT"]), "LANGUAGE"] =  "African"
pt_ids.loc[pt_ids['LANGUAGE'].isin(["PERS", "*FAR"]), "LANGUAGE"] =  "Middle Eastern"
# pt_ids.loc[~pt_ids['LANGUAGE'].isin(["ENGL", "SPAN"]), "LANGUAGE"] =  "OTHER"

pt_ids.loc[pt_ids['ETHNICITY'].isin(["ASIAN", "ASIAN - ASIAN INDIAN", "ASIAN - CHINESE",
                                                            "ASIAN - FILIPINO", "ASIAN - JAPANESE","ASIAN - OTHER",
                                                           ]), "ETHNICITY"] =  "ASIAN"
pt_ids.loc[pt_ids['ETHNICITY'].isin(["BLACK/AFRICAN AMERICAN", "BLACK/CAPE VERDEAN", "BLACK/HAITIAN", 
                                                           ]), "ETHNICITY"] =  "BLACK"
pt_ids.loc[pt_ids['ETHNICITY'].isin(["HISPANIC OR LATINO", "HISPANIC/LATINO - MEXICAN", 
                                                           "HISPANIC/LATINO - PUERTO RICAN", "HISPANIC/LATINO - SALVADORAN",  
                                                           ]), "ETHNICITY"] =  "HISPANIC/LATINO"
pt_ids.loc[pt_ids['ETHNICITY'].isin(["WHITE", "WHITE - OTHER EUROPEAN", "WHITE - RUSSIAN",   
                                                           ]), "ETHNICITY"] =  "WHITE"
pt_ids.loc[pt_ids['ETHNICITY'].isin(["PORTUGUESE",   
                                                           ]), "ETHNICITY"] =  "OTHER"
pt_ids.loc[pt_ids['ETHNICITY'].isin(["AMERICAN INDIAN/ALASKA NATIVE",   "AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE"
                                                           ]), "ETHNICITY"] =  "AMERICAN INDIAN"

In [47]:
mytable = TableOne(pt_ids, columns=columns, categorical=categorical, pval=False, groupby=groupby)
# print(mytable.tabulate(tablefmt = "fancy_grid"))
mytable.to_excel('n2c2_tableone_all.xlsx')

# with pd.option_context("display.max_rows", None):
#     display()

In [130]:
diags.merge(data, left_on="HADM_ID", right_on="HADM ID")['ICD9_CODE'].value_counts()

4280     2286
5849     2055
4019     2034
42731    1985
51881    1558
         ... 
29532       2
33520       2
4279        2
99642       2
6010        2
Name: ICD9_CODE, Length: 1562, dtype: int64

In [131]:
diags

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275
